In [ ]:
Fase 2 - Análise de Engajamento de Mídias Globo com Orientação a Objetos

In [ ]:
-> -> Estrutura de Diretórios <- <-

fase_2/
│
├── interacoes_globo.csv
├── main.py
│
├── entidades/
│   ├── __init__.py
│   ├── plataforma.py
│   ├── conteudo.py
│   ├── interacao.py
│   └── usuario.py
│
└── analise/
    ├── __init__.py
    └── sistema.py

In [ ]:
-> -> main.py <- <-

from analise.sistema import SistemaAnaliseEngajamento
#Classe principal que gerencia os relatórios em sistema.py
import os #Para verificar csv

def exibir_menu(): #Menu para Processamento de Dados e Seleção de Relatórios
    print("\n--- MENU PRINCIPAL ---")
    print("1. Processar arquivo CSV")
    print("2. Gerar Relatório de Engajamento por Conteúdo")
    print("3. Gerar Relatório de Atividade de Usuários")
    print("4. Top Conteúdos por Tempo Total Consumido")
    print("0. Sair")
    return input("Escolha uma opção: ")

"""
Menu simples a partir de While/True para organizar o acesso
aos relatórios solicitados na diretriz do projeto.
'O main.py deve instanciar SistemaAnaliseEngajamento,
carregar os dados e apresentar as métricas.'
"""

sistema = SistemaAnaliseEngajamento()

caminho_csv = "interacoes_globo.csv"
"""
Pode ser ampliado futuramente para acessar um arquivo específico
com o endereço do csv e assim não ficar preso a apenas interacoes_globo.csv.
"""
dados_processados = False
#Processamento será inicializado pelo usuário no menu.

print("\n\nFase 2: Análise de Engajamento de Mídias Globo com Orientação a Objetos\n")

while True:
    opcao = exibir_menu()

    if opcao == "1":
        if os.path.exists(caminho_csv):
            sistema.processar_interacoes_do_csv(caminho_csv)
            dados_processados = True
            print("\nDados do arquivo CSV processados.")
        else:
            print(f"Arquivo CSV não encontrado: {caminho_csv}")

    elif opcao == "2":
        if dados_processados:
            sistema.gerar_relatorio_engajamento_conteudos() #Em sistema.py
        else:
            print("Primeiro processe o arquivo CSV (Opção 1).")

    elif opcao == "3":
        if dados_processados:
            sistema.gerar_relatorio_atividade_usuarios() #Em sistema.py
        else:
            print("Primeiro processe o arquivo CSV (Opção 1).")

    elif opcao == "4":
        if dados_processados:
            sistema.gerar_relatorio_top_conteudos_consumidos(5) #Em sistema.py
        else:
            print("Primeiro processe o arquivo CSV (Opção 1).")

    elif opcao == "0":
        print("Encerrando o programa.")
        break
    else:
        print("Opção inválida. Tente novamente.")

In [ ]:
-> -> analise/sistema.py <- <-

import csv #Bilioteca interna para tratar csv
from entidades.usuario import Usuario #Recebe Classes e Funções de usuario.py
from entidades.plataforma import Plataforma #Recebe Classes e Funções de plataforma.py
from entidades.conteudo import Video, Podcast, Artigo #Recebe Classes e Funções de conteudo.py
from entidades.interacao import Interacao #Recebe Classes e Funções de interacao.py

"""
Como a Fase 2 é uma reformulação do código da Fase 1,
foi mantido a formatação dos outputs sempre que possível,
além de algumas outras padronizações.
"""

class SistemaAnaliseEngajamento:
#Classe que recebe os dados das entidades, organiza e analisa os dados
    VERSAO_ANALISE = "2.0"

    def __init__(self):
        # Atributos privados
        self.__plataformas_registradas = {}
        self.__conteudos_registrados = {}
        self.__usuarios_registrados = {}
        self.__proximo_id_plataforma = 1 # Contador para gerar IDs para plataformas

    def cadastrar_plataforma(self, nome_plataforma):
        #Cadastra uma nova plataforma se ela ainda não estiver registrada.
        if nome_plataforma not in self.__plataformas_registradas:
            nova = Plataforma(nome_plataforma, self.__proximo_id_plataforma)
            self.__plataformas_registradas[nome_plataforma] = nova
            self.__proximo_id_plataforma += 1
        return self.__plataformas_registradas[nome_plataforma]

    def obter_plataforma(self, nome_plataforma):
        #Carrega uma plataforma existente ou a cadastra se não existir.
        return self.__plataformas_registradas.get(nome_plataforma, self.cadastrar_plataforma(nome_plataforma))

    def listar_plataformas(self):
        #Retorna plataformas registradas.
        return list(self.__plataformas_registradas.values())

    def _carregar_interacoes_csv(self, caminho_arquivo):
    #Carrega os dados do csv como dicionários.
        lista = []
        try:
            with open(caminho_arquivo, mode='r', encoding='utf-8') as csvfile:
                leitor = csv.DictReader(csvfile)
                for linha in leitor:
                    lista.append(linha)
        except FileNotFoundError:
            print(f"Erro: O arquivo '{caminho_arquivo}' não foi encontrado.")
            return [] #Evita interrupção do código, retornando lista vazia.
        except Exception as e:
            print(f"Erro ao carregar CSV: {e}")
            return [] #Evita interrupção do código, retornando lista vazia.
        return lista

    
    def processar_interacoes_do_csv(self, caminho_arquivo):
    #Classe que processa os dicionários oriundos dos dados do csv
        linhas = self._carregar_interacoes_csv(caminho_arquivo)
        if not linhas:
            print("Nenhuma linha para processar ou erro ao carregar o CSV.")
            return

        for linha in linhas: 
            try:
                # Extrair dados da linha do CSV
                id_usuario = int(linha['id_usuario'])
                id_conteudo = int(linha['id_conteudo'])
                nome_conteudo = linha['nome_conteudo']
                timestamp = linha['timestamp_interacao']
                tipo = linha['tipo_interacao']

                valor_duracao = linha['watch_duration_seconds']
                
                #Caso 'watch_duration_seconds' seja vazio ou não numérico
                if not str(valor_duracao).strip().isdigit(): #Se o valor do csv sem espaços não for inteiro e positivo
                    duracao = 0
                else:
                    duracao = int(valor_duracao)
                    if duracao < 0:
                        duracao = 0

                
                comentario = linha['comment_text']
                nome_plataforma = linha['plataforma']

                # Carrega ou cadastra a plataforma
                plataforma = self.obter_plataforma(nome_plataforma)

                tipo_conteudo = None #Placeholder para futura coluna no csv com tipo de conteúdo
                
                # Carrega ou cadastra o conteúdo consumido.
                if id_conteudo not in self.__conteudos_registrados:
                    if tipo_conteudo == "podcast":
                        conteudo = Podcast(id_conteudo, nome_conteudo, 0)
                    elif tipo_conteudo == "artigo":
                        conteudo = Artigo(id_conteudo, nome_conteudo, 0)
                    else:
                        conteudo = Video(id_conteudo, nome_conteudo, 0)

                    self.__conteudos_registrados[id_conteudo] = conteudo
                else:
                    conteudo = self.__conteudos_registrados[id_conteudo]

                # Carrega ou cadastra o usuário
                if id_usuario not in self.__usuarios_registrados:
                    usuario = Usuario(id_usuario)
                    self.__usuarios_registrados[id_usuario] = usuario
                else:
                    usuario = self.__usuarios_registrados[id_usuario]

                # Cadastra Interacao e relaciona ao Conteudo e Usuario
                interacao = Interacao(id_usuario, timestamp, tipo, duracao, comentario, conteudo, plataforma)
                conteudo.adicionar_interacao(interacao)
                usuario.registrar_interacao(interacao) # Chamada para o novo método registrar_interacao
                
            except ValueError as ve:
                print(f"Erro de valor ao processar linha: {linha} - {ve}")
            except KeyError as ke:
                print(f"Erro: Coluna '{ke}' não encontrada no CSV na linha: {linha}")
            except Exception as e:
                print(f"Erro inesperado ao processar interação na linha {linha}: {e}")
        

    def gerar_relatorio_engajamento_conteudos(self, top_n=None):
    #Relatório de Engajamento por Conteúdo ou com maior quantidade de interações
        if not self.__conteudos_registrados:
            print("Nenhum conteúdo registrado para gerar relatório.")
            return

        print("\n-> -> RESULTADOS DE ENGAJAMENTO DE CONTEÚDOS <- <-\n")
        
        conteudos_para_relatorio = list(self.__conteudos_registrados.values())
        if top_n:
            # Ordena Conteúdo por total de Interações
            conteudos_para_relatorio = sorted(conteudos_para_relatorio, 
                                            key=lambda c: c.calcular_total_interacoes_engajamento(), 
                                            reverse=True)
            conteudos_para_relatorio = conteudos_para_relatorio[:top_n]


        for conteudo in conteudos_para_relatorio:
            print(f"ID: {conteudo.id_conteudo} - {conteudo.nome_conteudo}")
            print(f"Total de interações: {conteudo.calcular_total_interacoes_engajamento()}")
            
            contagem_tipos = conteudo.calcular_contagem_por_tipo_interacao()
            if contagem_tipos:
                print("Interações por tipo:")
                for tipo, qtd in contagem_tipos.items():
                    print(f"             {tipo}: {qtd}") #Identação Artificial estética, seguindo formatação da Fase 1

            tempo_total = conteudo.calcular_tempo_total_consumo()
            if tempo_total > 0:
                tempo_medio = conteudo.calcular_media_tempo_consumo()
                print(f"Tempo total assistido: {tempo_total} segundos ou {self.converter_segundos(tempo_total)}")
                print(f"Média de tempo assistido: {tempo_medio:.2f} segundos")
                """
                Tempo mostrado também no formato HH:MM:SS para facilitar ao usuário,
                seguindo o que foi feito na Fase 1
                """

            comentarios = conteudo.listar_comentarios()
            if comentarios:
                print(f"Quantidade de comentários: {len(comentarios)}")
                for idx, c in enumerate(comentarios):
                    print(f"Comentário {idx+1}: {c}")

            print("\n\n") 

    
    def gerar_relatorio_atividade_usuarios(self):
    #Relatório com todas as atividades de cada usuário
        if not self.__usuarios_registrados:
            print("Nenhum usuário registrado para gerar relatório.")
            return

        print("\n-> -> RESULTADOS DE ATIVIDADE DE USUÁRIOS <- <-\n")
        for usuario in self.__usuarios_registrados.values():

            print(f"Usuário (ID): {usuario.id_usuario}")
            
            #Número de Interações (total de interações, não apenas engajamento)
            #Agora acessando a property interacoes_realizadas
            print(f"Número de Interações: {len(usuario.interacoes_realizadas)}") 
            
            #Contagem por tipo de interação
            contagem = usuario.calcular_contagem_por_tipo_interacao()
            if contagem:
                print("Contagem por tipo de interação:")
                for tipo, qtd in contagem.items():
                    print(f"             {tipo}: {qtd}") 

            #Tempo total e médio de consumo (apenas se houver consumo)
            total_consumo = usuario.calcular_tempo_total_consumo()
            if total_consumo > 0: 
                media_consumo = usuario.calcular_media_tempo_consumo()
                print(f"Tempo total assistido: {total_consumo} segundos ou {self.converter_segundos(total_consumo)}") 
                print(f"Média de tempo assistido: {media_consumo:.2f} segundos") 

            
            #Comentários (apenas se houver comentários, com quantidade e numeração)
            comentarios = usuario.listar_comentarios()
            if comentarios:
                print(f"Quantidade de comentários: {len(comentarios)}") 
                for idx, c in enumerate(comentarios):
                    print(f"Comentário {idx+1}: {c}") 

            
            conteudos_unicos = usuario.obter_conteudos_unicos_consumidos()
            #Para adiconar chaves únicas ao dicionário de conteúdos consumidos por usuário
            if conteudos_unicos:
                print(f"Conteúdos únicos consumidos: {len(conteudos_unicos)}")

            
            plataformas_frequentes = usuario.plataformas_mais_frequentes(top_n=3)
            if plataformas_frequentes:
                print("Top 3 Plataformas Mais Frequentes:")
                for plat, cont in plataformas_frequentes:
                    print(f"             {plat.nome_plataforma}: {cont} interação(ões)")

            print("\n\n") 

    def gerar_relatorio_top_conteudos_consumidos(self, n=0):
    #Top 5 Conteudo mais consumidos
        if not self.__conteudos_registrados:
            print("Nenhum conteúdo registrado.") #Se não houver conteúdos registrados
            return

        ordenados = sorted(self.__conteudos_registrados.values(), key=lambda c: c.calcular_tempo_total_consumo(), reverse=True)
        #Função com lambda que ordena os conteúdos pelo tempo de consumo
        
        if n <= 0:
            top = ordenados  #Mostra todos os conteúdos.
        else:
            top = ordenados[:n]  #Mostra os top N indicados no main.

        print("\n-> -> TOP CONTEÚDOS POR TEMPO TOTAL CONSUMIDO <- <-\n")
        for idx, c in enumerate(top): #Enumera os Top N conteúdos por consumo
            tempo_total = c.calcular_tempo_total_consumo() #Super Classe em conteudo.py
            print(f"{idx+1}o. {c.nome_conteudo} ({self.converter_segundos(tempo_total)} consumidos)")
    
    
    def converter_segundos(self, total_segundos):
    #Função para Converter total de segundos do csv no formato HH:MM:SS
        if not isinstance(total_segundos, (int, float)) or total_segundos < 0:
            return "0:00:00" #Entradas inválidas

        horas = int(total_segundos // 3600)
        minutos = int((total_segundos % 3600) // 60)
        segundos = int(total_segundos % 60)

        return f"{horas}:{minutos:02}:{segundos:02}"

In [ ]:
-> -> entidades/conteudo.py <- <-
 
from entidades.plataforma import Plataforma #Importa classes de plataforma.py

class Conteudo: #Classe Base para diversos Conteúdos. Super Classe de Video, Podcast e Artigo
    def __init__(self, id_conteudo, nome_conteudo):
        self._id_conteudo = id_conteudo
        self._nome_conteudo = nome_conteudo
        self._interacoes = [] #Lista para armazenar as interações associadas a este conteúdo
        

    @property
    def id_conteudo(self):
        return self._id_conteudo
        

    @property
    def nome_conteudo(self):
        return self._nome_conteudo
        

    def adicionar_interacao(self, interacao): #Adiciona interação a lista do Conteúdo
        self._interacoes.append(interacao)
        

    def calcular_total_interacoes_engajamento(self):
    #Calcula o total de Interações de Engajamento (Like, Share e Comment)
        total = 0
        for i in self._interacoes:
            if i.tipo_interacao in ["like", "share", "comment"]:
                total += 1
        return total
        

    def calcular_contagem_por_tipo_interacao(self): #Calcula o total de interações por tipo 
        contagem = {}
        for i in self._interacoes:
            tipo = i.tipo_interacao
            if tipo not in contagem:
                contagem[tipo] = 0
            contagem[tipo] += 1
        return contagem
        

    def calcular_tempo_total_consumo(self): #Calcula o tempo total de consumo do conteúdo.
        total = 0
        for i in self._interacoes:
            if isinstance(i.watch_duration_seconds, int) and i.watch_duration_seconds > 0:
                total += i.watch_duration_seconds #Watch_duration_seconds: Inteiro e Positivo
        return total
        

    def calcular_media_tempo_consumo(self): #Calcula o tempo médio de consumo do conteúdo.
        duracoes = []
        for i in self._interacoes:
            d = i.watch_duration_seconds
            if isinstance(d, int) and d > 0:
                duracoes.append(d)
        if len(duracoes) > 0:
            return sum(duracoes) / len(duracoes)
        else:
            return 0
    #Padronização desde Projeto 1: int e floats vazia retorna zero, strings vazias retorna none.
    

    def listar_comentarios(self): #Lista todos os comentários associados ao conteúdo.
        comentarios = []
        for i in self._interacoes:
            c = i.comment_text
            if c is None or c.strip() != "": # Adiciona apenas se o comentário não for vazio
                comentarios.append(c)
        return comentarios

    def __str__(self):
        return "Conteúdo ID " + str(self._id_conteudo) + ": " + self._nome_conteudo

    def __repr__(self):
        return self.__str__()
        
        

class Video(Conteudo): #Tipo de conteúdo analisado pelos dados do csv
    def __init__(self, id_conteudo, nome_conteudo, duracao_total_video_seg):
        super().__init__(id_conteudo, nome_conteudo)
        self.__duracao_total_video_seg = duracao_total_video_seg

    @property
    def duracao_total_video_seg(self):
        return self.__duracao_total_video_seg

    def calcular_percentual_medio_assistido(self): #Método Exclusivo
        if self.__duracao_total_video_seg == 0:
            return 0
        media = self.calcular_media_tempo_consumo()
        return (media / self.__duracao_total_video_seg) * 100
        

class Podcast(Conteudo): #Sem dados para análise
    def __init__(self, id_conteudo, nome_conteudo, duracao_total_episodio_seg=0):
        super().__init__(id_conteudo, nome_conteudo)
        self.__duracao_total_episodio_seg = duracao_total_episodio_seg

    @property
    def duracao_total_episodio_seg(self):
        return self.__duracao_total_episodio_seg
        

class Artigo(Conteudo): #Sem dados para análise
    def __init__(self, id_conteudo, nome_conteudo, tempo_leitura_estimado_seg=0):
        super().__init__(id_conteudo, nome_conteudo)
        self.__tempo_leitura_estimado_seg = tempo_leitura_estimado_seg

    @property
    def tempo_leitura_estimado_seg(self):
        return self.__tempo_leitura_estimado_seg

"""
As classes Podcast e Artigo são solicitadas nas diretrizes do projeto,
mas não é solicitado nenhum relatório sobre eles já que a origem dos
dados (interacoes_globo.csv) não apresenta nenhuma informação sobre
consumo de podcasts e/ou artigos. Portanto, as classes foram
modeladas para serem escalonadas numa futura refatoração do projeto
Fase_2.py, assim como está sendo feito do Fase_1.py para o Fase_2.py
"""

In [ ]:
-> -> entidades/plataforma.py <- <-
 
class Plataforma:
    def __init__(self, nome_plataforma, id_plataforma=None):
        if not nome_plataforma or not nome_plataforma.strip():
            raise ValueError("Nome da plataforma não pode ser vazio.")
        self.__nome_plataforma = nome_plataforma.strip()
        self.__id_plataforma = id_plataforma

    @property
    def nome_plataforma(self):
        return self.__nome_plataforma

    @nome_plataforma.setter #Padronização do argumento de todos os setters como "valor"
    def nome_plataforma(self, valor):
        if not valor or not valor.strip():
            raise ValueError("Nome da plataforma não pode ser vazio.")
        self.__nome_plataforma = valor.strip()

    @property
    def id_plataforma(self):
        return self.__id_plataforma

    @id_plataforma.setter
    def id_plataforma(self, valor):
        self.__id_plataforma = valor

    def __str__(self):
        return self.__nome_plataforma

    def __repr__(self): #Retorna Plataforma(nome='...') conforme solicitado no Projeto.
        return f"Plataforma(nome='{self.__nome_plataforma}')"

    def __eq__(self, other): #Compara duas plataformas pelo nome.
        if isinstance(other, Plataforma):
            return self.__nome_plataforma == other.__nome_plataforma
        return False

    def __hash__(self): #hash do nome da plataforma
        return hash(self.__nome_plataforma)


In [ ]:
-> -> entidades/usuario.py <- <-
 
from collections import Counter

class Usuario: #Cada um dos usuários, suas interações e métricas
    def __init__(self, id_usuario):
        # Atributos privados conforme diretrizes
        self.__id_usuario = id_usuario
        self.__interacoes_realizadas = [] # Lista de objetos Interacao

    @property
    def id_usuario(self):
        return self.__id_usuario

    @property
    def interacoes_realizadas(self):
        return self.__interacoes_realizadas

    def registrar_interacao(self, interacao): 
    #Adiciona uma interação à lista de interações realizadas pelo usuário
        self.__interacoes_realizadas.append(interacao)

    def obter_interacoes_por_tipo(self, tipo_desejado: str) -> list: 
    #Retorna uma lista de um tipo de interação.
        return [i for i in self.__interacoes_realizadas if i.tipo_interacao == tipo_desejado]

    def obter_conteudos_unicos_consumidos(self) -> set: 
    #Retorna set de conteúdos únicos consumidos pelo usuário.
        conteudos = set()
        for interacao in self.__interacoes_realizadas:
            if interacao.conteudo_associado:
                conteudos.add(interacao.conteudo_associado)
        return conteudos

    def calcular_tempo_total_consumo_plataforma(self, plataforma) -> int: #Tempo total de consumo de um usuário em uma plataforma
        total_tempo = 0
        for interacao in self.__interacoes_realizadas:
            if interacao.plataforma_interacao == plataforma:
                if isinstance(interacao.watch_duration_seconds, int) and interacao.watch_duration_seconds > 0: #Inteiro e Positivo
                    total_tempo += interacao.watch_duration_seconds
        return total_tempo

    def plataformas_mais_frequentes(self, top_n=3) -> list:
    #3 Plataformas onde o usuário mais interage - Tupla (Plataforma, Contagem)
        plataformas = []
        for interacao in self.__interacoes_realizadas:
            if interacao.plataforma_interacao:
                plataformas.append(interacao.plataforma_interacao)
        
        contagem_plataformas = Counter(plataformas) #Counter para Frequência
        
        top_plataformas = contagem_plataformas.most_common(top_n) #Ordenar por quantidade
        return top_plataformas


    def calcular_total_interacoes_engajamento(self):
    #Total de interações de engajamento (like, share, comment) do usuário.
        total = 0
        for i in self.__interacoes_realizadas:
            if i.tipo_interacao in ["like", "share", "comment"]:
                total += 1
        return total

    def calcular_contagem_por_tipo_interacao(self):
    #Quantidade de interações por tipo para o usuário.
        contagem = {}
        for i in self.__interacoes_realizadas:
            tipo = i.tipo_interacao
            if tipo not in contagem:
                contagem[tipo] = 0
            contagem[tipo] += 1
        return contagem

    def calcular_tempo_total_consumo(self):
        total = 0
        for i in self.__interacoes_realizadas:
            if isinstance(i.watch_duration_seconds, int) and i.watch_duration_seconds > 0:
                total += i.watch_duration_seconds
        return total

    def calcular_media_tempo_consumo(self):
        duracoes = []
        for i in self.__interacoes_realizadas:
            d = i.watch_duration_seconds
            if isinstance(d, int) and d > 0:
                duracoes.append(d)
        if duracoes:
            return sum(duracoes) / len(duracoes)
        else:
            return 0

    def listar_comentarios(self):
        comentarios = []
        for i in self.__interacoes_realizadas:
            c = i.comment_text
            if c: 
                comentarios.append(c)
        return comentarios

    def __str__(self):
        return f"Usuário ID {self.__id_usuario}"

    def __repr__(self):
        return self.__str__()


In [ ]:
-> -> entidades/interacao.py <- <-
 
from datetime import datetime
from entidades.plataforma import Plataforma
from entidades.conteudo import Conteudo

class Interacao: #Uma interação de um usuário um conteúdo em uma plataforma.
    TIPOS_INTERACAO_VALIDOS = {"view_start", "like", "share", "comment", "vote_bbb"}
    __proximo_id = 1 # Contador para gerar IDs únicos para interações

    def __init__(self, id_usuario, timestamp, tipo_interacao, watch_duration_seconds=0, comment_text="", conteudo_associado=None, plataforma_interacao=None):
        self.__interacao_id = Interacao.__proximo_id
        Interacao.__proximo_id += 1

        self.__id_usuario = int(id_usuario)

        try:
            self.__timestamp_interacao = datetime.fromisoformat(timestamp)
        except ValueError:
            self.__timestamp_interacao = datetime.min 
        
        self.__tipo_interacao = tipo_interacao if tipo_interacao in self.TIPOS_INTERACAO_VALIDOS else "view_start"
        #Se inválido, usa "view_start" como interação minima.

        try:
            self.__watch_duration_seconds = int(watch_duration_seconds)
            if self.__watch_duration_seconds < 0:
                self.__watch_duration_seconds = 0
        except (ValueError, TypeError): 
            self.__watch_duration_seconds = 0 #Duração inválida

        self.__comment_text = comment_text.strip() if comment_text else ""
        self.__conteudo_associado = conteudo_associado
        self.__plataforma_interacao = plataforma_interacao

    @property
    def interacao_id(self):
        return self.__interacao_id

    @property
    def conteudo_associado(self):
        return self.__conteudo_associado

    @property
    def plataforma_interacao(self):
        return self.__plataforma_interacao

    @property
    def id_usuario(self):
        return self.__id_usuario

    @property
    def timestamp_interacao(self):
        # Retorna o objeto datetime
        return self.__timestamp_interacao

    @property
    def tipo_interacao(self):
        return self.__tipo_interacao

    @property
    def watch_duration_seconds(self):
        return self.__watch_duration_seconds

    @property
    def comment_text(self):
        return self.__comment_text

    def __str__(self):
        return f"Interação {self.__interacao_id}: {self.__tipo_interacao} por usuário {self.__id_usuario} em {self.conteudo_associado.nome_conteudo}"

    def __repr__(self):
        return (f"Interacao(id={self.__interacao_id}, usuario={self.__id_usuario}, tipo='{self.__tipo_interacao}', "
                f"duracao={self.__watch_duration_seconds}, comentario='{self.__comment_text}')")